In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

In [2]:
nx = 5
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()

250


In [2]:
from ase.io import read
structure = read("POSCAR_bcc2",format='vasp')
#structure = read("sigma5_310.data",style='atomic',format='lammps-data')

sys = pc.System()
sys.read_inputfile(structure,format='ase')

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [3]:
sys.box = box
sys.atoms = satoms

In [4]:
sys.find_neighbors(method='voronoi', cutoff=0.1)

In [5]:
len(sys.atoms["vertex_positions_unique"])

1500

In [7]:
sys.atom.voronoi.vertex.positions

[[[-1.5635, 4.440892098500626e-16, -0.7817499999999996],
  [1.5634999999999997, 2.7755575615628914e-16, -0.7817499999999996],
  [-4.440892098500626e-16, 1.5635000000000001, 0.7817499999999993],
  [1.6653345369377348e-16, 1.5635, -0.7817499999999995],
  [2.7755575615628914e-16, -0.7817499999999995, 1.5635],
  [0.7817499999999992, -4.440892098500626e-16, 1.5635],
  [1.5634999999999994, -5.551115123125783e-16, 0.7817499999999995],
  [-0.7817499999999995, 1.5635000000000001, 4.440892098500626e-16],
  [0.7817499999999992, 1.5635, -4.440892098500626e-16],
  [1.5634999999999994, 0.7817499999999994, -4.440892098500626e-16],
  [-1.5635, 4.440892098500626e-16, 0.7817500000000002],
  [-4.440892098500626e-16, 0.7817499999999993, 1.5635000000000001],
  [-0.7817499999999995, 4.440892098500626e-16, 1.5635000000000001],
  [0.7817500000000003, -1.5634999999999994, 3.885780586188048e-16],
  [1.5634999999999997, -0.7817499999999997, 3.3306690738754696e-16],
  [0.0, 0.7817500000000004, -1.563499999999999]

Get unique vertices

In [23]:
vertex_positions = np.vstack(sys.atoms["vertex_positions"])
vertex_positions.shape

(6000, 3)

In [14]:
vor = Voronoi(sys.positions)

In [20]:
vor.vertices[2]

array([12.508  ,  1.5635 ,  0.78175])

In [ ]:
abs(a-b) <= max( rel_tol * max(abs(a), abs(b)), abs_tol )

In [4]:
sys.to_file("a1.dat")

In [5]:
sys.repeat([0.5,0.5,0.5])

250
displaced 250
[15.634999999999998, 0, 0]
[31.27  0.    0.  ]
500
displaced 500
[0, 15.634999999999998, 0]
[ 0.   31.27  0.  ]
1000
displaced 1000
[0, 0, 15.634999999999998]
[ 0.    0.   31.27]
here
[array([31.27,  0.  ,  0.  ]), array([ 0.  , 31.27,  0.  ]), array([ 0.  ,  0.  , 31.27])]
[[15.634999999999998, 0, 0], [0, 15.634999999999998, 0], [0, 0, 15.634999999999998]]


In [6]:
sys.to_file("a2.dat")

In [16]:
a = bulk('Mo')
sys = pc.System()
sys.read_inputfile(a, format="ase")

In [18]:
sys.box

array([[-1.575,  1.575,  1.575],
       [ 1.575, -1.575,  1.575],
       [ 1.575,  1.575, -1.575]])

In [22]:
[-1.575, -1.575, -1.575]

[-1.575, -1.575, -1.575]

In [17]:
sys.boxdims

[8.183940065762943, 8.183940065762943, 8.183940065762943]

In [23]:
pcm.remap_atom_into_box([-1.575, -1.575, -1.575], 
    1, 
    sys.rot, 
    sys.rotinv,
    sys.boxdims)

[3.1499999999999995, 3.1499999999999995, 3.1499999999999995]

In [26]:
datadict = {key:[] for key in sys.atoms.keys()}

In [27]:
datadict

{'positions': [],
 'ids': [],
 'types': [],
 'species': [],
 'ghost': [],
 'mask_1': [],
 'mask_2': [],
 'condition': [],
 'head': []}

Try a fully python numpy based algorithm for finding nearest neighbor

In [10]:
def get_distance(x, box, dim=3):
    box_2 = box/2
    xdum = []
    for i in range(dim):
        xd = np.meshgrid(x[i], x[i])[1] - np.meshgrid(x[i], x[i])[0]
        xdum.append(xd)
    xd = np.array(xdum)
    for i in range(dim):
        xd[i] = np.where(xd[i] > box_2[i], xd[i]-box[i], xd[i])
        xd[i] = np.where(xd[i] < -box_2[i], xd[i]+box[i], xd[i])
    return xd

In [12]:
x = sys.atoms['positions']
box = sys.boxdims

In [16]:
x = np.array(x)

In [20]:
y = x.T

In [28]:
%%timeit
xd = get_distance(y, np.array(box))

224 ms ± 857 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
xd.shape

(3, 2000, 2000)

In [23]:
np.where(xd[]

array([[[ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        [ 1.5635,  0.    ,  1.5635, ...,  3.127 ,  4.6905,  3.127 ],
        [ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        ...,
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ],
        [-3.127 , -4.6905, -3.127 , ..., -1.5635,  0.    , -1.5635],
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ]],

       [[ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        [ 1.5635,  0.    ,  1.5635, ...,  3.127 ,  4.6905,  3.127 ],
        [ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        ...,
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ],
        [-3.127 , -4.6905, -3.127 , ..., -1.5635,  0.    , -1.5635],
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ]],

       [[ 0.    , -1.5635, -3.127 , ...,  4.6905,  3.127 ,  1.5635],
        [ 1.5635,  0.    , -1.5635, ...,  6.254 ,  4.6905,  3.127 ],
    